In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
!curl -O https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3832  100  3832    0     0   4553      0 --:--:-- --:--:-- --:--:--  4594


## Importations

In [2]:
import pandas as pd
import minsearch
import os
from groq import Groq
from dotenv import load_dotenv

In [3]:
# Load environment variables from .env file
load_dotenv()
GROQ_API_KEY="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"

## Data Loading

In [4]:
file_path = 'data/data.csv'
df = pd.read_csv(file_path)

In [5]:
df.head()

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."


## Ingestion

In [19]:
documents =df.to_dict(orient='records')

In [10]:
documents

[{'id': 0,
  'exercise_name': 'Push-Ups',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Upper Body',
  'type': 'Push',
  'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
  'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'},
 {'id': 1,
  'exercise_name': 'Squats',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Stand with feet shoulder-width apart. Lower your body as if sitting back into a chair, keeping your chest up. Return to standing.'},
 {'id': 2,
  'exercise_name': 'Plank',
  'type_of_activity': 'Strength/Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Core',
  'type': 'Hold',
  'muscle_groups_activated': 'Rectus Abdominis, Transvers

In [20]:
Index=minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields=[]
)

In [21]:
Index.fit(documents)

## RAG FLOW

In [22]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [23]:
def search(query):
    boost = {}

    results = Index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=5
    )

    return results

In [24]:
entry_template = """
exercise_name: {exercise_name}
type_of_activity: {type_of_activity}
type_of_equipment: {type_of_equipment}
body_part: {body_part}
type: {type}
muscle_groups_activated: {muscle_groups_activated}
instructions: {instructions}
""".strip()

prompt_template = """
You're a fitness insrtuctor. Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.
QUESTION: {question}

CONTEXT: 
{context}
""".strip()


def build_prompt(query, search_results):

    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc)+ "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
def llm(prompt):
    response = client.chat.completions.create(
        model='llama3-8b-8192',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
query="glute"

In [28]:
search_results = search(query)
prompt =build_prompt(query,search_results)
answer=rag(query)
print(answer)

Based on the context, the answers to the question about the "glute" is:

* The exercises that target the glutes are: Glute Kickback, Glute Bridge, and Single-Leg Glute Bridge.
* These exercises work the muscle groups: Glutes, Hamstrings.

Let me know if you have any further questions!


## Retrieval Evaluation

In [7]:
df_questions=pd.read_csv("data/ground-truth-retrieval.csv")

In [42]:
df_questions.head()

,id,questions
0,0,What type of activity is a push-up exercise?
1,0,What equipment is needed for push-ups?
2,0,Which part of the body does the push-up exerci...
3,0,"What type of push-up is this, and what is its ..."
4,0,Which muscle groups are activated during push-...


In [9]:
ground_truth=df_questions.to_dict(orient='records')

In [10]:
ground_truth[0]

{'id': 0, 'questions': 'What type of activity is a push-up exercise?'}

In [31]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [32]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


In [38]:
def minsearch_search(query):
    boost = {}

    results = Index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results


In [39]:

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [40]:
from tqdm.auto import tqdm

evaluate(ground_truth, lambda q: minsearch_search(q['questions']))


  0%|          | 0/1041 [00:00<?, ?it/s]

{'hit_rate': 0.8837656099903939, 'mrr': 0.752562783038288}

## Firnding the best parameters

In [45]:
df_validation = df_questions[:100]
df_test = df_questions[100:]

In [46]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [47]:
gt_val = df_validation.to_dict(orient='records')


In [54]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = Index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [55]:
param_ranges = {
    'exercise_name': (0.0, 3.0),
    'type_of_activity': (0.0, 3.0),
    'type_of_equipment': (0.0, 3.0),
    'body_part': (0.0, 3.0),
    'type': (0.0, 3.0),
    'muscle_groups_activated': (0.0, 3.0),
    'instructions': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['questions'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [56]:
simple_optimize(param_ranges, objective, n_iterations=20)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'exercise_name': 2.342708684219849,
  'type_of_activity': 0.4789400617826105,
  'type_of_equipment': 0.8363498052146746,
  'body_part': 1.7987554763033087,
  'type': 1.84476010174011,
  'muscle_groups_activated': 0.601553368824103,
  'instructions': 0.5438817514127631},
 0.8086785714285715)

In [59]:
def minsearch_improved(query):
    boost = {
        'exercise_name': 2.11,
        'type_of_activity': 1.46,
        'type_of_equipment': 0.65,
        'body_part': 2.65,
        'type': 1.31,
        'muscle_groups_activated': 2.54,
        'instructions': 0.74
    }

    results = Index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['questions']))

  0%|          | 0/1041 [00:00<?, ?it/s]

{'hit_rate': 0.878001921229587, 'mrr': 0.8052433557476789}

## RAG EVALUATION

In [40]:
prompt_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [11]:
len(ground_truth)

1041

In [35]:
record=ground_truth[0]
question=record["questions"]
answer_llm=rag(question)


In [41]:
prompt=prompt_template.format(question=question,answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What type of activity is a push-up exercise?
Generated Answer: Based on the CONTEXT, a push-up exercise is a type of Strength activity that can be performed using Bodyweight equipment, targeting the Upper Body (specifically, Pectorals, Triceps, and/or Deltoids).

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [44]:
df_sample = df_questions.sample(n=200, random_state=1)
sample = df_sample.to_dict(orient='records')


In [ ]:
from tqdm.auto import tqdm
import json

evaluations = []

for record in tqdm(sample):
    id=record['id']
    if id in evaluations:
        continue
    question = question
    answer_llm = answer_llm 

    prompt = prompt_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

In [ ]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [ ]:
df_eval.relevance.value_counts(normalize=True)


In [ ]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)


In [ ]:
evaluations_gpt4o = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question, model='gpt-4o') 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)
    
    evaluations_gpt4o.append((record, answer_llm, evaluation))

In [ ]:
df_eval = pd.DataFrame(evaluations_gpt4o, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [ ]:
df_eval.relevance.value_counts()


In [ ]:
df_eval.relevance.value_counts(normalize=True)
